In [4]:
import pandas as pd
data_clean = pd.read_csv('TAClean2.csv', encoding='latin1')
data_clean.head()

,Ulasan,SENTIMEN
0,sesuai gambar bagus,POSITIF
1,barang sesuai pesan manfaat,POSITIF
2,toko amanah hp sesuai deskripsi,POSITIF
3,alhamdulillah tablet ganti cek bagus terima ka...,POSITIF
4,kirim cepat barang fungsi warna sesuai pilih r...,POSITIF


In [5]:
data_clean = data_clean.astype({'SENTIMEN' : 'category'})
data_clean = data_clean.astype({'Ulasan' : 'string'})
data_clean.dtypes

Ulasan      string[python]
SENTIMEN          category
dtype: object

In [6]:
# PROSES TF IDF
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data_clean['Ulasan'].astype('U'))

tf = TfidfVectorizer()
text_tf = tf.fit_transform(data_clean['Ulasan'].astype('U'))
print(text_tf)

  (0, 71)	0.39784416191672173
  (0, 320)	0.8281025738363363
  (0, 876)	0.394925499346923
  (1, 563)	0.7966751746822
  (1, 727)	0.4296320302800328
  (1, 97)	0.264982525961549
  (1, 876)	0.3324293090832699
  (2, 234)	0.5661743018117791
  (2, 380)	0.4066940035961517
  (2, 35)	0.46848503645691997
  (2, 996)	0.42865378615454836
  (2, 876)	0.33290291323178156
  (3, 325)	0.398126578552004
  (3, 529)	0.32272699997435217
  (3, 186)	0.152493038370299
  (3, 809)	0.3038836602189331
  (3, 458)	0.14669265360582337
  (3, 970)	0.1421929805227773
  (3, 185)	0.3885361817639921
  (3, 324)	0.4881016302088219
  (3, 942)	0.33029412805628106
  (3, 26)	0.2317331895433447
  (3, 71)	0.16735435097438275
  (4, 685)	0.3036332382180207
  (4, 530)	0.3036332382180207
  :	:
  (654, 970)	0.5151286859995052
  (654, 97)	0.23986379741828448
  (654, 876)	0.30091703654987534
  (655, 294)	0.3703452517454036
  (655, 89)	0.5038216434138417
  (655, 653)	0.20379024652130143
  (655, 689)	0.29012076120357
  (655, 34)	0.20992382008

In [7]:
#cosine similarity
from sklearn.metrics.pairwise import cosine_similarity
cos_sim=cosine_similarity(text_tf, text_tf)

print(cos_sim)

[[1.         0.13128481 0.13147185 ... 0.11883981 0.         0.        ]
 [0.13128481 1.         0.11066669 ... 0.16359336 0.         0.04371164]
 [0.13147185 0.11066669 1.         ... 0.10017616 0.339819   0.11422409]
 ...
 [0.11883981 0.16359336 0.10017616 ... 1.         0.29761092 0.21932968]
 [0.         0.         0.339819   ... 0.29761092 1.         0.2473917 ]
 [0.         0.04371164 0.11422409 ... 0.21932968 0.2473917  1.        ]]


In [8]:
def init_position(lb, ub, N, dim):
    X = np.zeros([N, dim], dtype='float')
    for i in range(N):
        for d in range(dim):
            X[i,d] = lb[0,d] + (ub[0,d] - lb[0,d]) * rand()        
    
    return X

In [9]:
def init_velocity(lb, ub, N, dim):
    V    = np.zeros([N, dim], dtype='float')
    Vmax = np.zeros([1, dim], dtype='float')
    Vmin = np.zeros([1, dim], dtype='float')
    # Maximum & minimum velocity
    for d in range(dim):
        Vmax[0,d] = (ub[0,d] - lb[0,d]) / 2
        Vmin[0,d] = -Vmax[0,d]
        
    for i in range(N):
        for d in range(dim):
            V[i,d] = Vmin[0,d] + (Vmax[0,d] - Vmin[0,d]) * rand()
        
    return V, Vmax, Vmin

In [10]:
def binary_conversion(X, thres, N, dim):
    Xbin = np.zeros([N, dim], dtype='int')
    for i in range(N):
        for d in range(dim):
            if X[i,d] > thres:
                Xbin[i,d] = 1
            else:
                Xbin[i,d] = 0
    
    return Xbin

In [11]:
def boundary(x, lb, ub):
    if x < lb:
        x = lb
    if x > ub:
        x = ub
    
    return x

In [12]:
# error rate
def error_rate(xtrain, ytrain, x, opts):
    # parameters
    k     = opts['k']
    fold  = opts['fold']
    xt    = fold['xt']
    yt    = fold['yt']
    xv    = fold['xv']
    yv    = fold['yv']
    
    # Number of instances
    num_train = np.size(xt, 0)
    num_valid = np.size(xv, 0)
    # Define selected features
    xtrain  = xt[:, x == 1]
    ytrain  = yt.reshape(num_train)  # Solve bug
    xvalid  = xv[:, x == 1]
    yvalid  = yv.reshape(num_valid)  # Solve bug   
    # Training
    mdl     = KNeighborsClassifier(n_neighbors = k)
    mdl.fit(xtrain, ytrain)
    # Prediction
    ypred   = mdl.predict(xvalid)
    acc     = np.sum(yvalid == ypred) / num_valid
    error   = 1 - acc
    
    return error

In [13]:
def model_selected_feature(xtrain,xtest,ytrain, ytest,k,fmdl,sf,fs_method):  
    # model with selected features
    num_train = np.size(xtrain, 0)
    num_valid = np.size(xtest, 0)
    x_train   = xtrain[:, sf]
    y_train   = ytrain.reshape(num_train)  # Solve bug
    x_valid   = xtest[:, sf]
    y_valid   = ytest.reshape(num_valid)  # Solve bug

    mdl       = KNeighborsClassifier(n_neighbors = k) 
    print("Training Accuracy: \n")
    mdl.fit(x_train, y_train)
    # Calculate the accuracy of the training model
    train_res = mdl.score(x_train, y_train)
    print("Training Accuracy: ", train_res)
    # Predict on dataset which model has not seen before
    
    mdl.predict(x_valid)
    # Calculate the accuracy of the model
    test_res = mdl.score(x_valid, y_valid)
    print("Testing Accuracy: \n", test_res)
    
    accuracy_convergence(mdl,x_valid,y_valid,num_valid,fmdl,fs_method,opts)
    #print(sum(train_res-test_res))

In [14]:
def accuracy_convergence(mdl,x_valid,y_valid,num_valid,fmdl,fs_method,opts):
    # accuracy
    y_pred    = mdl.predict(x_valid)
    Acc       = np.sum(y_valid == y_pred)  / num_valid
    print("Accuracy:", 100 * Acc)

    # number of selected features
    num_feat = fmdl['nf']
    print("Feature Size:", num_feat)

    # plot convergence
    curve   = fmdl['c']
    curve   = curve.reshape(np.size(curve,1))
    x       = np.arange(0, opts['T'], 1.0) + 1.0

    fig, ax = plt.subplots()
    ax.plot(x, curve, 'o-')
    ax.set_xlabel('Number of Iterations')
    ax.set_ylabel('Fitness')
    ax.set_title(fs_method)
    ax.grid()
    plt.show()

In [15]:
def pso_fs(xtrain, ytrain, opts):
    # Parameters
    ub    = 1
    lb    = 0
    thres = 0.5
    w     = 0.9    # inertia weight
    c1    = 2      # acceleration factor
    c2    = 2      # acceleration factor
    
    N        = opts['N']
    max_iter = opts['T']
    if 'w' in opts:
        w    = opts['w']
    if 'c1' in opts:
        c1   = opts['c1']
    if 'c2' in opts:
        c2   = opts['c2'] 
    
    # Dimension
    dim = np.size(xtrain, 1)
    if np.size(lb) == 1:
        ub = ub * np.ones([1, dim], dtype='float')
        lb = lb * np.ones([1, dim], dtype='float')
        
    # Initialize position & velocity
    X             = init_position(lb, ub, N, dim)
    V, Vmax, Vmin = init_velocity(lb, ub, N, dim) 
    
    # Pre
    fit   = np.zeros([N, 1], dtype='float')
    Xgb   = np.zeros([1, dim], dtype='float')
    fitG  = float('inf')
    Xpb   = np.zeros([N, dim], dtype='float')
    fitP  = float('inf') * np.ones([N, 1], dtype='float')
    curve = np.zeros([1, max_iter], dtype='float') 
    t     = 0
    
    while t < max_iter:
        # Binary conversion
        Xbin = binary_conversion(X, thres, N, dim)
        
        # Fitness
        for i in range(N):
            fit[i,0] = error_rate(xtrain, ytrain, Xbin[i,:], opts)
            if fit[i,0] < fitP[i,0]:
                Xpb[i,:]  = X[i,:]
                fitP[i,0] = fit[i,0]
            if fitP[i,0] < fitG:
                Xgb[0,:]  = Xpb[i,:]
                fitG      = fitP[i,0]
        
        # Store result
        curve[0,t] = fitG.copy()
        print("Iteration:", t + 1)
        print("Best (PSO):", curve[0,t])
        t += 1
        
        for i in range(N):
            for d in range(dim):
                # Update velocity
                r1     = rand()
                r2     = rand()
                V[i,d] = w * V[i,d] + c1 * r1 * (Xpb[i,d] - X[i,d]) + c2 * r2 * (Xgb[0,d] - X[i,d]) 
                # Boundary
                V[i,d] = boundary(V[i,d], Vmin[0,d], Vmax[0,d])
                # Update position
                X[i,d] = X[i,d] + V[i,d]
                # Boundary
                X[i,d] = boundary(X[i,d], lb[0,d], ub[0,d])
    
                 # Best feature subset
    Gbin       = binary_conversion(Xgb, thres, 1, dim) 
    Gbin       = Gbin.reshape(dim)
    pos        = np.asarray(range(0, dim))    
    sel_index  = pos[Gbin == 1]
    num_feat   = len(sel_index)
    # Create dictionary
    pso_data = {'sf': sel_index, 'c': curve, 'nf': num_feat}
    
    return pso_data 
    

In [16]:
def pso_fs_model(opts):
    fs_method='PSO KICH'

    # perform feature selection
    fmdl = pso_fs(feat, label, opts)
    sf   = fmdl['sf']
    print('\nSelected features are : ',sf)

    model_selected_feature(xtrain,xtest,ytrain, ytest,k, fmdl, sf,fs_method)

In [17]:
#splitting data split test 60:40
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(cos_sim, data_clean['SENTIMEN'], test_size=0.2, random_state=33)
print(X_test)
print(y_test)

pos = (y_test == 'POSITIF').sum()
neg = (y_test == 'NEGATIF').sum()
postrain = (y_train == 'POSITIF').sum()
negtrain = (y_train == 'NEGATIF').sum()
total = pos + neg
print("Jumlah data uji dengan sentimen positif:", pos)
print("Jumlah data uji dengan sentimen negatif:",neg)
print("Jumlah data latih dengan sentimen positif:", postrain)
print("Jumlah data latih dengan sentimen negatif:",negtrain)
data_clean['SENTIMEN'].value_counts()

[[0.         0.         0.339819   ... 0.29761092 1.         0.2473917 ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.07052512 0.07010445 ... 0.         0.35946634 0.04358027]
 ...
 [0.         0.02074803 0.09760895 ... 0.01878124 0.         0.01291634]
 [0.03397878 0.0286017  0.1568849  ... 0.02589044 0.05665839 0.        ]
 [0.0755441  0.14447373 0.         ... 0.03603897 0.         0.15077602]]
575    POSITIF
342    POSITIF
337    POSITIF
318    POSITIF
571    POSITIF
        ...   
298    POSITIF
608    POSITIF
87     POSITIF
104    NEGATIF
427    POSITIF
Name: SENTIMEN, Length: 132, dtype: category
Categories (2, object): ['NEGATIF', 'POSITIF']
Jumlah data uji dengan sentimen positif: 125
Jumlah data uji dengan sentimen negatif: 7
Jumlah data latih dengan sentimen positif: 499
Jumlah data latih dengan sentimen negatif: 26


SENTIMEN
POSITIF    624
NEGATIF     33
Name: count, dtype: int64

In [18]:
# perform algoritma KNN
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=7).fit(X_train, y_train)
predicted = clf.predict(X_test)
print(f'confusion matrix:\n {confusion_matrix(y_test, predicted)}')
print('===============================================\n')
tn, fp, fn, tp = confusion_matrix(y_test, predicted).ravel()
print("TN:", tn)
print("FP:", fp)
print("FN:", fn)
print("TP:", tp)
print(classification_report(y_test, predicted, zero_division=0))
print('===============================================\n')
print("Hasil Klasifikasi Sentimen Analisis Tokopedia:")
print("Accuracy:" , accuracy_score(y_test,predicted))
print("Precision:" , precision_score(y_test,predicted, average="binary", pos_label="POSITIF"))
print("Recall:" , recall_score(y_test,predicted, average="binary", pos_label="POSITIF"))
print("f1_score:" , f1_score(y_test,predicted, average="binary", pos_label="POSITIF"))
print("error_rate:", 1-accuracy_score(y_test,predicted))

confusion matrix:
 [[  0   7]
 [  0 125]]

TN: 0
FP: 7
FN: 0
TP: 125
              precision    recall  f1-score   support

     NEGATIF       0.00      0.00      0.00         7
     POSITIF       0.95      1.00      0.97       125

    accuracy                           0.95       132
   macro avg       0.47      0.50      0.49       132
weighted avg       0.90      0.95      0.92       132


Hasil Klasifikasi Sentimen Analisis Tokopedia:
Accuracy: 0.946969696969697
Precision: 0.946969696969697
Recall: 1.0
f1_score: 0.9727626459143969
error_rate: 0.05303030303030298


In [19]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=5)
cross_val_score(clf, cos_sim, data_clean['SENTIMEN'], cv=10)

array([0.95454545, 0.89393939, 0.92424242, 0.93939394, 0.93939394,
       0.86363636, 0.93939394, 0.96923077, 0.95384615, 0.96923077])

In [20]:
cross_val_score(clf, cos_sim, data_clean['SENTIMEN'], cv=10).mean()

0.9346853146853148